# Limiting Program Efficiency: The Impact of Excessive Threads and Processes

In software development, particularly in applications requiring concurrent execution, threads and processes are fundamental units of execution. They enable multitasking within an application, allowing for multiple operations to run simultaneously or in parallel, thus potentially improving the program's overall efficiency and responsiveness. However, the relationship between the number of threads/processes and program efficiency is not linear. Beyond a certain point, increasing the number of threads or processes can actually degrade performance, making the application slower or less responsive. Understanding this relationship is crucial for optimizing application performance.


## The Problem with Too Many Threads or Processes

**Resource Competition**: Each thread or process consumes system resources, such as CPU time and memory. As the number of threads/processes increases, they compete for these limited resources, leading to context switching overhead and increased memory usage, which can slow down the application.

**Context Switching Overhead**: Context switching occurs when the CPU switches from executing one thread to another. This involves saving the state of the current thread and loading the state of the next thread, which consumes CPU cycles. With too many threads, the overhead from context switching can significantly reduce the CPU time available for actual work, degrading performance.

**Synchronization and Deadlocks**: Multithreading and multiprocessing require careful synchronization, especially when threads/processes share data or resources. Incorrect synchronization can lead to deadlocks or race conditions, causing the application to slow down or even crash. More threads/processes increase the complexity of synchronization, raising the likelihood of such issues.


## Effective Use of Thread/Process Pools

To mitigate the issues associated with excessive threading/multiprocessing, using a thread or process pool with a maximum limit is an effective strategy. This approach involves creating a pool of worker threads/processes and reusing them to execute tasks, rather than creating new ones for each task. Key benefits include:

**Optimized Resource Utilization**: By limiting the number of concurrent threads/processes, a pool ensures that system resources are not overwhelmed. This optimization prevents excessive context switching and memory usage, maintaining the application's responsiveness.

**Balanced Load**: A thread/process pool can dynamically allocate tasks to workers based on their availability, ensuring a more balanced load across the system. This prevents some threads from being overloaded while others remain idle, improving overall efficiency.

**Reduced Overhead**: Reusing threads/processes eliminates the overhead associated with creating and destroying them for each task. This reduction in overhead can lead to significant performance improvements, especially in applications that handle many short-lived tasks.

**Simplified Synchronization**: Managing a fixed number of threads/processes can simplify synchronization mechanisms, reducing the risk of deadlocks and race conditions. This simplification makes the application more stable and reliable.


### Example

Below is the same code from the previous multi-processing module.

👆 Press the play button next to the code below to run it. Nothing will happen but
it will initialize the `download_and_process_image()` function and `IMAGE_URLS`
variable so it can be used by the cells below.

In [ ]:
from PIL import Image, ImageFilter
import requests
from PIL import Image
from io import BytesIO
from PIL import Image, ImageFilter
import os


# URLs for ultra high res images from NASA
IMAGE_URLS = [
    "https://mars.nasa.gov/system/downloadable_items/48800_PIA26242-Ingenuitys_View_of_Sand_Dunes_During_Flight_70.jpg",
    "https://mars.nasa.gov/system/downloadable_items/48811_ZCAM_SOL1052_R0_ZCAM05175_INGENUITY_MDI_E01.png",
    "https://mars.nasa.gov/system/downloadable_items/48730_PIA26205-FigC.png",
    "https://mars.nasa.gov/system/downloadable_items/48642_PIA26202.jpg",
    "https://mars.nasa.gov/system/downloadable_items/48431_PIA25968-FigA.jpg",
    "https://mars.nasa.gov/system/downloadable_items/48217_PIA25830-FigureA.jpg",
    "https://stsci-opo.org/STScI-01HN3AEK5XYSQEC92Y37V1AGYV.tif",
    "https://stsci-opo.org/STScI-01EVVGKMPMHDMK28W6VHFAB2FN.png",
    "https://stsci-opo.org/STScI-01FVYYAGQT81SXBGNJX96BF7RB.tif",
    "https://stsci-opo.org/STScI-01EVT0H99TZK40KGNK18M6EK9E.tif",
    "https://stsci-opo.org/STScI-01EVT0GQ9ZABZZZ90G19AR2SHQ.tif",
    "https://stsci-opo.org/STScI-01EVT0WKMTHPCZ68YEG0Y9WPFG.tif",
    "https://stsci-opo.org/STScI-01EVVBGGRWCXT9WM0EF5XXAXPK.tif",
    "https://stsci-opo.org/STScI-01EVVH0Z1XVJ2T61FQ5XRYV9HK.tif",
    "https://stsci-opo.org/STScI-01EVVH2H1HX8AV52T63T7Z20CA.tif"]


def download_and_process_image(url):
    # Download the image
    print(f"Downloading {url}")
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    # Perform CPU-intensive tasks
    print(f"Processing {url}")
    # Convert image to grayscale
    img = img.convert("L")
    # Apply a blur filter
    img = img.filter(ImageFilter.GaussianBlur(10))
    # Scale the image to 128x128 pixels
    img = img.resize((128, 128), resample=Image.LANCZOS)

    # make and output dir for the images
    os.makedirs("processed-images", exist_ok=True)

    # Save the processed image as PNG
    img_name = f"./processed-images/processed_{url.split('/')[-1].split('.')[0]}.png"
    img.save(img_name, 'PNG')
    print(f"Saved {img_name}")


### Thread Pool Example

This code snippet creates a thread pool using `concurrent.futures.ThreadPoolExecutor` with a maximum of 5 worker threads to concurrently download and process images from a list named `IMAGE_URLS`. Each image URL in the list is submitted to the executor as a task, where the `download_and_process_image` function is called asynchronously for each URL. The total execution time for downloading and processing all images is calculated and printed at the end, demonstrating the efficiency gained through concurrent execution.

In [ ]:
import concurrent.futures
import time

start_time = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:

    # Submit tasks to the executor
    for url in IMAGE_URLS:
        executor.submit(download_and_process_image, url)

end_time = time.time()
print(f"Process completed in {end_time - start_time} seconds.")


### Process Pool Example

This code snippet uses `concurrent.futures.ProcessPoolExecutor` to create a pool of processes, with up to 5 processes running concurrently, to download and process images from a list named `IMAGE_URLS`. Each image URL is submitted as a task to the executor, where the `download_and_process_image` function is executed in a separate process for each URL, leveraging multiple CPU cores for parallel execution. The total time taken to complete the downloading and processing of all images is calculated and printed, showcasing the performance improvement through parallel processing.

In [ ]:
import concurrent.futures
import time

start_time = time.time()
with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:

    # Submit tasks to the executor
    for url in IMAGE_URLS:
        executor.submit(download_and_process_image, url)

end_time = time.time()
print(f"Process completed in {end_time - start_time} seconds.")



## Conclusion

While threads and processes are powerful tools for improving the performance of concurrent applications, their misuse can lead to inefficiencies and decreased performance. Understanding the limitations and overheads associated with threading and multiprocessing is crucial. Implementing a thread/process pool with a maximum number of workers can help manage these challenges, optimizing resource utilization and ensuring that the application remains responsive and efficient.